<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from notebooks.utils.functions import *
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Getting the current date
today = datetime.date.today()
  
# Yesterday date
yesterday = today - datetime.timedelta(days = 1)

country="germany"
city="reutlingen"
street="Zaisentalstraße"
station_id="A54451"
# Read from .aqi_key
AQI_API_KEY=open(".aqi_key").read()
str(yesterday)

'2024-11-04'

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [5]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157271
Connected. Call `.close()` to terminate connection gracefully.


---

## <span style='color:#ff5f27'> 🌫 Get Yesterday's Air Quality data (PM2.5) from the Feature Group</span>

In [6]:
# Read all data from the feature group
df_air_quality = air_quality_fg.read()

df = pd.DataFrame()
df_air_quality['day'] = pd.to_datetime(df_air_quality['date']).dt.date

row = df_air_quality.loc[df_air_quality['day'] == yesterday]

pm25_yesterday = row['pm25'].iloc[0]
pm25_yesterday

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.07s) 


6.42

In [7]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city", "street"]].groupby("street").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

In [8]:
import requests
import pandas as pd

# The API endpoint URL
# url = f"https://api.waqi.info/feed/{country}/{street}/?token={AQI_API_KEY}"
url = f"https://api.waqi.info/feed/{station_id}/?token={AQI_API_KEY}"

# Make a GET request to fetch the data from the API
response = requests.get(url)

print(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the JSON content from the response
    data = response.json()
    
    # Check if the API response contains the data
    if data['status'] == 'ok':
        # Extract the air quality data
        aqi_data = data['data']
        aq_today_df = pd.DataFrame()  
        aq_today_df['pm25'] = [aqi_data['iaqi'].get('pm25', {}).get('v', None)]
        aq_today_df['pm25'] = aq_today_df['pm25'].astype('float32')
    else:
        print("Error: The API response does not contain data.")
        print(response.json())
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    
if pm25_yesterday is None: 
    print(f"Couldn't find yesterday's pm25 in your feature group. The latest value is from {df_air_quality['day'].max()}")
    # It is hard to get historical air quality data using aqicn.org, so we will just set it to today's value
    aq_today_df['pm25_yesterday'] = aq_today_df['pm25']
else:
    aq_today_df['pm25_yesterday'] = pm25_yesterday

aq_today_df['city'] = country
aq_today_df['street'] = street
aq_today_df['date'] = today
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])

aq_today_df

https://api.waqi.info/feed/A54451/?token=ba5ae24c6b6960a19efb8e95617369d34872c83b


,pm25,pm25_yesterday,city,street,date
0,14.0,6.42,germany,Zaisentalstraße,2024-11-05


In [9]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pm25            1 non-null      float32       
 1   pm25_yesterday  1 non-null      float32       
 2   city            1 non-null      object        
 3   street          1 non-null      object        
 4   date            1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float32(2), object(2)
memory usage: 164.0+ bytes


## <span style='color:#ff5f27'> 🌦 Filling gaps in Weather data</span>

In [10]:
# Read all data from the feature group
df_weather = weather_fg.read()
latest_date_weather_df = df_weather[["date", "city"]].groupby("city").max()
latest_date_weather = latest_date_weather_df['date'].iloc[0].date()
weather_df = get_historical_weather(city, str(latest_date_weather), str(today))
weather_df
weather_df = weather_df.dropna()
weather_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 
Coordinates 48.47100067138672°N 9.075630187988281°E
Elevation 385.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-03,7.402583,0.0,7.895416,121.260376,reutlingen


In [11]:
## Get weather forecst and write to feature group

hourly_df = get_hourly_weather_forecast(city)
hourly_df = hourly_df.set_index('date')
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df

Coordinates 48.75°N 9.25°E
Elevation 385.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m
0,2024-11-05,12.40,0.0,2.189795,80.537750
1,2024-11-06,11.90,0.0,2.811690,230.194473
2,2024-11-07,11.45,0.0,6.924738,98.972549
3,2024-11-08,10.25,0.0,3.319036,102.528801
4,2024-11-09,9.15,0.0,2.902413,352.875061
5,2024-11-10,6.80,0.0,3.219938,26.564985
6,2024-11-11,4.95,0.0,3.877318,21.801476
7,2024-11-12,4.45,0.0,9.826088,331.557159
8,2024-11-13,5.05,0.1,17.873556,62.402794
9,2024-11-14,6.20,0.0,15.463246,65.224876


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [12]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-05 14:35:48,915 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157271/fs/1147974/fg/1336668


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157271/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2021988a4d0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 999.9,
           "strict_min": true
         },
         "meta": {
           "expectationId": 659460
         }
       },
       "result": {
         "observed_value": 14.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-05T01:35:48.000915Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [13]:
# Insert new data
weather_fg.insert(weather_df)

2024-11-05 14:36:12,376 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157271/fs/1147974/fg/1336669


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157271/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x20219b8bc10>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 659461
         }
       },
       "result": {
         "observed_value": 7.895416259765625,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-05T01:36:12.000376Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_m

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
